# 인트로

맵을 사용하면 데이터프레임 혹은 시리즈의 데이터를 전체 열에 대해 한 번에 한 값씩 변환할 수 있습니다. 그러나 데이터를 그룹화한 다음 데이터가 속한 그룹에 특정 작업을 수행하는 경우가 많습니다.

배우게 되겠지만, 이는 `groupby()` 연산자를 이용해 할 수 있습니다. 또한 데이터를 정렬하는 방법과 더불어 데이터프레임을 인덱싱하는 보다 복잡한 방법과 같은 몇 가지 추가 항목도 다룰 예정입니다.

# 그룹별 분석

우리가 지금까지 많이 사용해 온 함수 중 하나는 `value_counts()`입니다. 다음을 수행하여 `value_counts()`의 기능을 따라할 수 있습니다.

In [1]:
import pandas as pd
reviews = pd.read_csv("../../data/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

In [2]:
reviews.groupby('points').points.count()

points
80     397
81     692
      ... 
99      33
100     19
Name: points, Length: 21, dtype: int64

`groupby()`는 같은 점수가 매겨진 와인끼리 묶여진 그룹을 생성하였습니다. 그 후에 각 그룹별 `points()` 열을 선택해 그것이 몇 번 나타났는지 세었습니다. `value_counts()`는 이 `groupby()` 연산의 쉬운 방법일 뿐입니다.

우리는 이 데이터에 이전에 사용했던 요약 함수를 적용할 수 있습니다. 예를 들어, 각 점수 카테고리에서 가장 저렴한 와인을 얻으려면 다음을 수행할 수 있습니다:

In [3]:
reviews.groupby('points').price.min()

points
80      5.0
81      5.0
       ... 
99     44.0
100    80.0
Name: price, Length: 21, dtype: float64

생성된 그룹은 값이 일치하는 데이터만 포함하는 데이터프레임의 슬라이스라고 생각할 수 있습니다. 이 데이터프레임은 `apply()` 메소드를 이용해 직접 접근할 수 있으며, 원하는 방식으로 데이터를 조작할 수 있습니다. 예를 들어, 데이터셋의 각 포도주 양조장(winery)에서 첫 번째로 리뷰된 와인 이름을 선택하려면 다음과 같이 할 수 있습니다:

In [4]:
reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

보다 세밀한 제어를 위해 두 개 이상의 열로 그룹화할 수도 있습니다. 예를 들어, 국가별, 지방별로 최고의 와인을 선택하는 방법은 다음과 같습니다:

In [5]:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
...                               ...   
Uruguay   San Jose            Uruguay   
          Uruguay             Uruguay   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
...                                                                       ...   
Uruguay   San Jose          Baked, sweet, heavy aromas turn earthy with ti...   
          Uruguay           Cherry and berry aromas are ripe, healthy and ...   

                                          designation  points  price  \
country   province                                                     
Argentina Mendoza Province           Nicasia Vineyard      97  120.0   
          Other                               Reserva      95   90.0   
...                                               ...     ...    ...   
Uruguay   San Jose           El Preciado Gran Reserva      87   50.0   
          Uruguay           Blend 002 Limited Edition      91   22.0   

                                    province region_1 region_2  \
country   province                                               
Argentina Mendoza Province  Mendoza Province  Mendoza      NaN   
          Other                        Other    Salta      NaN   
...                                      ...      ...      ...   
Uruguay   San Jose                  San Jose      NaN      NaN   
          Uruguay                    Uruguay      NaN      NaN   

                                  taster_name taster_twitter_handle  \
country   province                                                    
Argentina Mendoza Province  Michael Schachner           @wineschach   
          Other             Michael Schachner           @wineschach   
...                                       ...                   ...   
Uruguay   San Jose          Michael Schachner           @wineschach   
          Uruguay           Michael Schachner           @wineschach   

                                                                        title  \
country   province                                                              
Argentina Mendoza Province  Bodega Catena Zapata 2006 Nicasia Vineyard Mal...   
          Other                            Colomé 2010 Reserva Malbec (Salta)   
...                                                                       ...   
Uruguay   San Jose          Castillo Viejo 2005 El Preciado Gran Reserva R...   
          Uruguay           Narbona NV Blend 002 Limited Edition Tannat-Ca...   

                                          variety                winery  
country   province                                                       
Argentina Mendoza Province                 Malbec  Bodega Catena Zapata  
          Other                            Malbec                Colomé  
...                                           ...                   ...  
Uruguay   San Jose                      Red Blend        Castillo Viejo  
          Uruguay           Tannat-Cabernet Franc               Narbona  

[425 rows x 13 columns]

언급할 가치가 있는 또 다른 `groupby()` 메소드는 `agg()`로, 데이터프레임에 대해 여러 함수를 동시에 실행할 수 있습니다. 예를 들어, 다음과 같이 데이터셋의 간단한 통계 요약을 생성할 수 있습니다:

In [6]:
reviews.groupby(['country']).price.agg([len, min, max])

,len,min,max
country,,,
Argentina,3800,4.0,230.0
Armenia,2,14.0,15.0
...,...,...,...
Ukraine,14,6.0,13.0
Uruguay,109,10.0,130.0


`groupby()`를 효과적으로 사용하면 데이터셋으로 많은 강력한 작업을 수행할 수 있습니다.

# 다중 인덱스

지금까지 살펴본 모든 예제에서는 단일 레이블 인덱스를 사용하여 데이터프레임 또는 시리즈에 대해 작업했습니다. `groupby()`는 실행하는 연산에 따라 다중 인덱스라고 불리는 결과가 나오기도 한다는 점에서 약간 다릅니다.

다중 인덱스는 여러 단계가 있다는 점에서 일반 인덱스와 다릅니다. 예를 들어 다음과 같습니다:

In [7]:
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  3264
          Other              536
...                          ...
Uruguay   San Jose             3
          Uruguay             24

[425 rows x 1 columns]

In [8]:
mi = countries_reviewed.index
type(mi)

pandas.core.indexes.multi.MultiIndex

다중 인덱스에는 단일 인덱스에 없는 계층화된 구조를 처리하는 여러 가지 메소드가 있습니다. 또한 값을 검색하려면 두 단계의 레이블이 필요합니다. 일반적으로 다중 인덱스를 다루는 것은 판다스를 처음 접한 사용자들에게 새로운 경험입니다.

다중 인덱스 사용 사례는 판다스 설명서의 [MultiIndex / Advanced Selection](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#multiindex-advanced-indexing) 섹션에 자세히 설명되어 있습니다.

그러나 일반적으로 가장 자주 사용하는 다중 인덱스 메소드는 일반 인덱스로 다시 변환하는 `reset_index()` 메소드입니다:

In [9]:
countries_reviewed.reset_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


# 정렬

다시 `countries_reviewed`를 보면 그룹화가 값 순서가 아닌 인덱스 순으로 데이터를 반환한다는 것을 알 수 있습니다. 즉, `groupby`의 결과를 출력할 때 행의 순서는 데이터가 아닌 인덱스의 값에 따라 달라집니다.

데이터를 원하는 순서대로 얻기 위해 직접 정렬할 수 있습니다. `sort_values()` 메소드가 이에 유용합니다.

In [10]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
179,Greece,Muscat of Kefallonian,1
192,Greece,Sterea Ellada,1
...,...,...,...
415,US,Washington,8639
392,US,California,36247


`sort_values()`는 기본적으로 가장 낮은 값이 먼저 나오는 오름차순 정렬로 설정됩니다. 그러나 대부분의 경우 우리는 높은 숫자가 먼저 나오는 내림차순 정렬을 원합니다. 이는 다음과 같이 할 수 있습니다:

In [11]:
countries_reviewed.sort_values(by='len', ascending=False)

,country,province,len
392,US,California,36247
415,US,Washington,8639
...,...,...,...
63,Chile,Coelemu,1
149,Greece,Beotia,1


인덱스 값을 기준으로 정렬하려면 `sort_index()` 메소드를 사용합니다. 메소드의 인수와 기본 순서는 동일합니다:

In [12]:
countries_reviewed.sort_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


마지막으로, 한 번에 둘 이상의 열을 기준으로 정렬할 수 있습니다:

In [13]:
countries_reviewed.sort_values(by=['country', 'len'])

,country,province,len
1,Argentina,Other,536
0,Argentina,Mendoza Province,3264
...,...,...,...
424,Uruguay,Uruguay,24
419,Uruguay,Canelones,43
